## OBJECT DETECTION BY SEGMENTATION
## Langkah:
    - Akuisisi dataset
    - Pemodelan Latar Belakang
    - Deteksi dan segmentasi
    - Ekstraksi dan analisa

In [ ]:

def read_ppm(filepath):
    """Membaca file gambar format P3 PPM, dengan kemampuan mengabaikan komentar."""
    with open(filepath, 'r') as f:
        lines = f.readlines()
    
    # --- BAGIAN BARU: Membersihkan baris komentar ---
    cleaned_lines = []
    for line in lines:
        # Hapus spasi di awal/akhir, jika baris tidak dimulai dengan '#', simpan
        if line.strip() and not line.strip().startswith('#'):
            cleaned_lines.append(line.strip())
    # -----------------------------------------------
    
    assert cleaned_lines[0] == 'P3'
    width, height = map(int, cleaned_lines[1].split())
    max_val = int(cleaned_lines[2])
    
    # Menggabungkan semua sisa baris data piksel menjadi satu string besar
    pixel_data = ' '.join(cleaned_lines[3:])
    pixels_flat = [int(p) for p in pixel_data.split()]
    
    image = []
    idx = 0
    for _ in range(height):
        row = []
        for _ in range(width):
            # Ambil 3 nilai (R, G, B)
            r, g, b = pixels_flat[idx], pixels_flat[idx+1], pixels_flat[idx+2]
            row.append((r, g, b))
            idx += 3
        image.append(row)
        
    return image, width, height

def write_ppm(filepath, image, width, height):
    """Menyimpan gambar ke format P3 PPM."""
    with open(filepath, 'w') as f:
        f.write('P3\n')
        f.write(f'{width} {height}\n')
        f.write('255\n')
        for row in image:
            for r, g, b in row:
                f.write(f'{r} {g} {b} ')
            f.write('\n')

# --- Core Object Detection Logic ---

def to_grayscale(image):
    """Mengubah gambar warna menjadi grayscale."""
    gray_image = []
    for row in image:
        gray_row = [int(0.299 * r + 0.587 * g + 0.114 * b) for r, g, b in row]
        gray_image.append(gray_row)
    return gray_image

def create_background_model(image_paths):
    """Membuat model latar belakang dari beberapa gambar."""
    if not image_paths:
        return None
    
    # Baca gambar pertama untuk mendapatkan dimensi
    sample_image, width, height = read_ppm(image_paths[0])
    
    # Inisialisasi sum_image dengan nol
    sum_image = [[0] * width for _ in range(height)]
    
    # Akumulasi nilai piksel
    num_images = len(image_paths)
    for path in image_paths:
        img, _, _ = read_ppm(path)
        gray_img = to_grayscale(img)
        for r in range(height):
            for c in range(width):
                sum_image[r][c] += gray_img[r][c]
    
    # Hitung rata-rata
    background_model = [[int(sum_image[r][c] / num_images) for c in range(width)] for r in range(height)]
    return background_model, width, height

def background_subtraction(current_frame_gray, background_model, threshold):
    """Melakukan background subtraction dan binarisasi."""
    height, width = len(current_frame_gray), len(current_frame_gray[0])
    foreground_mask = [[0] * width for _ in range(height)]
    
    for r in range(height):
        for c in range(width):
            diff = abs(current_frame_gray[r][c] - background_model[r][c])
            if diff > threshold:
                foreground_mask[r][c] = 255  # Putih (Foreground)
            else:
                foreground_mask[r][c] = 0    # Hitam (Background)
    return foreground_mask

def connected_components_labeling(mask):
    """Menemukan dan melabeli objek yang terhubung."""
    height, width = len(mask), len(mask[0])
    labels = [[0] * width for _ in range(height)]
    current_label = 1
    
    for r in range(height):
        for c in range(width):
            if mask[r][c] == 255 and labels[r][c] == 0:
                # Mulai BFS dari piksel ini
                q = [(r, c)]
                labels[r][c] = current_label
                
                head = 0
                while head < len(q):
                    row, col = q[head]
                    head += 1
                    
                    # Cek tetangga (atas, bawah, kiri, kanan)
                    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                        nr, nc = row + dr, col + dc
                        if 0 <= nr < height and 0 <= nc < width and \
                           mask[nr][nc] == 255 and labels[nr][nc] == 0:
                            labels[nr][nc] = current_label
                            q.append((nr, nc))
                current_label += 1
    return labels, current_label - 1

def extract_bounding_boxes(labels, num_labels, min_area=50):
    """Mengekstrak bounding box dari label, dengan filter area minimum."""
    if num_labels == 0:
        return []
        
    boxes = {}
    for i in range(1, num_labels + 1):
        boxes[i] = {'min_r': float('inf'), 'min_c': float('inf'), 
                    'max_r': float('-inf'), 'max_c': float('-inf'), 'area': 0}

    height, width = len(labels), len(labels[0])
    for r in range(height):
        for c in range(width):
            label = labels[r][c]
            if label > 0:
                boxes[label]['min_r'] = min(boxes[label]['min_r'], r)
                boxes[label]['min_c'] = min(boxes[label]['min_c'], c)
                boxes[label]['max_r'] = max(boxes[label]['max_r'], r)
                boxes[label]['max_c'] = max(boxes[label]['max_c'], c)
                boxes[label]['area'] += 1
    
    # Filter bounding box yang terlalu kecil
    final_boxes = []
    for label, box in boxes.items():
        if box['area'] > min_area:
            final_boxes.append((box['min_c'], box['min_r'], box['max_c'], box['max_r']))
            
    return final_boxes

def draw_boxes_on_image(image, boxes):
    """Menggambar bounding box pada gambar asli."""
    img_copy = [row[:] for row in image]
    for x1, y1, x2, y2 in boxes:
        # Garis horizontal
        for c in range(x1, x2 + 1):
            if 0 <= y1 < len(img_copy) and 0 <= c < len(img_copy[0]):
                img_copy[y1][c] = (255, 0, 0) # Merah
            if 0 <= y2 < len(img_copy) and 0 <= c < len(img_copy[0]):
                img_copy[y2][c] = (255, 0, 0)
        # Garis vertikal
        for r in range(y1, y2 + 1):
            if 0 <= r < len(img_copy) and 0 <= x1 < len(img_copy[0]):
                img_copy[r][x1] = (255, 0, 0)
            if 0 <= r < len(img_copy) and 0 <= x2 < len(img_copy[0]):
                img_copy[r][x2] = (255, 0, 0)
    return img_copy

def read_ground_truth_xml(filepath):
    """
    Membaca file anotasi .xml sederhana untuk mengekstrak bounding box ground truth.
    Fungsi ini membaca file sebagai teks biasa untuk menghindari library eksternal.
    """
    boxes = []
    try:
        with open(filepath, 'r') as f:
            lines = f.readlines()
        
        current_box = {}
        for line in lines:
            line = line.strip()
            if '<xmin>' in line:
                current_box['xmin'] = int(line.replace('<xmin>', '').replace('</xmin>', ''))
            elif '<ymin>' in line:
                current_box['ymin'] = int(line.replace('<ymin>', '').replace('</ymin>', ''))
            elif '<xmax>' in line:
                current_box['xmax'] = int(line.replace('<xmax>', '').replace('</xmax>', ''))
            elif '<ymax>' in line:
                current_box['ymax'] = int(line.replace('<ymax>', '').replace('</ymax>', ''))
            
            # Jika satu box sudah lengkap, tambahkan ke daftar dan reset
            if 'xmin' in current_box and 'ymin' in current_box and 'xmax' in current_box and 'ymax' in current_box:
                # Format box: (x_min, y_min, x_max, y_max)
                boxes.append((current_box['xmin'], current_box['ymin'], current_box['xmax'], current_box['ymax']))
                current_box = {}
                
    except FileNotFoundError:
        print(f"PERINGATAN: File ground truth tidak ditemukan di {filepath}")
        return []
    
    return boxes

def calculate_iou(box_a, box_b):
    """
    Menghitung Intersection over Union (IoU) antara dua bounding box.
    Format box: (x_min, y_min, x_max, y_max)
    """
    # Tentukan koordinat (x, y) dari area perpotongan (intersection)
    x_a = max(box_a[0], box_b[0])
    y_a = max(box_a[1], box_b[1])
    x_b = min(box_a[2], box_b[2])
    y_b = min(box_a[3], box_b[3])

    # Hitung luas area perpotongan
    intersection_area = max(0, x_b - x_a) * max(0, y_b - y_a)

    # Hitung luas masing-masing bounding box
    box_a_area = (box_a[2] - box_a[0]) * (box_a[3] - box_a[1])
    box_b_area = (box_b[2] - box_b[0]) * (box_b[3] - box_b[1])
    
    # Hitung luas gabungan (union)
    union_area = float(box_a_area + box_b_area - intersection_area)

    # Hitung IoU
    if union_area == 0:
        return 0.0
    
    iou = intersection_area / union_area
    return iou

def evaluate_detections(predicted_boxes, gt_boxes, iou_threshold=0.5):
    """
    Mengevaluasi deteksi dengan membandingkan prediksi dengan ground truth.
    Menghitung True Positives (TP), False Positives (FP), dan False Negatives (FN).
    """
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    # Salin list ground truth agar kita bisa menandai yang sudah terdeteksi
    gt_matches = [False] * len(gt_boxes)
    
    # Iterasi melalui setiap box yang diprediksi
    for pred_box in predicted_boxes:
        best_iou = 0
        best_gt_idx = -1
        
        # Cari ground truth box yang paling cocok untuk prediksi ini
        for i, gt_box in enumerate(gt_boxes):
            iou = calculate_iou(pred_box, gt_box)
            if iou > best_iou:
                best_iou = iou
                best_gt_idx = i
        
        # Jika IoU terbaik melebihi threshold dan GT box itu belum cocok,
        # maka ini adalah True Positive
        if best_iou >= iou_threshold and not gt_matches[best_gt_idx]:
            true_positives += 1
            gt_matches[best_gt_idx] = True # Tandai GT ini sebagai sudah terdeteksi
        else:
            # Jika tidak, ini adalah False Positive
            false_positives += 1
            
    # Semua ground truth yang tidak pernah terdeteksi adalah False Negative
    false_negatives = len(gt_boxes) - sum(gt_matches)
    
    return true_positives, false_positives, false_negatives

# --- Main Execution ---
if __name__ == "__main__":
    # --- 1. KONFIGURASI ---
    # Di sinilah Anda mengatur semua path dan parameter.
    
    # Tentukan path folder Anda
    folder_latar_belakang = 'dataset/train/'
    folder_test = 'dataset/test/'
    folder_output = 'output/' 

    # Cukup ubah nama file ini untuk menguji gambar yang berbeda
    test_image_name = 'test (1)' 

    # --- PERBAIKAN DIMULAI DI SINI ---

    # Path akan dibuat secara otomatis berdasarkan 'test_image_name'
    background_frame_paths = [folder_latar_belakang + f'frame ({i}).ppm' for i in range(1, 101)]
    test_frame_path = folder_test + test_image_name + '.ppm'
    
    # BENAR: Membangun path XML dari nama file dasar
    ground_truth_path = folder_test + test_image_name + '.xml' 
    
    # BENAR: Membuat nama file output yang bersih
    output_path = folder_output + f'hasil_{test_image_name}.ppm'

    # --- AKHIR PERBAIKAN ---

    # Parameter untuk algoritma
    SUBTRACTION_THRESHOLD = 35
    MIN_OBJECT_AREA = 150
    IOU_THRESHOLD = 0.5
    
    # --- 2. PROSES DETEKSI OBJEK ---
    print("--- MEMULAI PROSES DETEKSI ---")
    
    print("Membuat model latar belakang...")
    bg_model, width, height = create_background_model(background_frame_paths)
    print("Model latar belakang selesai dibuat.")

    print(f"Memproses frame: {test_frame_path}")
    test_image, _, _ = read_ppm(test_frame_path)
    test_image_gray = to_grayscale(test_image)
    
    foreground_mask = background_subtraction(test_image_gray, bg_model, SUBTRACTION_THRESHOLD)
    labels, num_labels = connected_components_labeling(foreground_mask)
    predicted_boxes = extract_bounding_boxes(labels, num_labels, MIN_OBJECT_AREA)
    print(f"Ditemukan {len(predicted_boxes)} objek setelah filter area.")
    
    final_image = draw_boxes_on_image(test_image, predicted_boxes)
    write_ppm(output_path, final_image, width, height)
    print(f"Hasil deteksi visual disimpan di: {output_path}")

    # --- 3. ANALISA PERFORMA ---
    print("\n--- MEMULAI ANALISA PERFORMA ---")
    
    ground_truth_boxes = read_ground_truth_xml(ground_truth_path)
    
    if not ground_truth_boxes:
        print("Analisa dilewati karena data ground truth tidak ditemukan atau kosong.")
    else:
        print(f"Ditemukan {len(ground_truth_boxes)} objek ground truth.")
        
        total_iou = 0
        matches = 0
        for pred_box in predicted_boxes:
            best_iou = 0
            for gt_box in ground_truth_boxes:
                iou = calculate_iou(pred_box, gt_box)
                if iou > best_iou:
                    best_iou = iou
            if best_iou > 0:
                total_iou += best_iou
                matches += 1
        
        avg_iou = total_iou / matches if matches > 0 else 0.0
        print(f"\nRata-rata IoU untuk deteksi yang cocok: {avg_iou:.4f}")

        tp, fp, fn = evaluate_detections(predicted_boxes, ground_truth_boxes, IOU_THRESHOLD)

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        
        print(f"\nHasil Evaluasi (IoU Threshold = {IOU_THRESHOLD}):")
        print(f"  - True Positives (TP) : {tp}")
        print(f"  - False Positives (FP): {fp}")
        print(f"  - False Negatives (FN): {fn}")
        print(f"\n  - Precision : {precision:.4f}")
        print(f"  - Recall    : {recall:.4f}")


--- MEMULAI PROSES DETEKSI ---
Membuat model latar belakang...
Model latar belakang selesai dibuat.
Memproses frame: dataset/test/test (1).ppm
Ditemukan 1 objek setelah filter area.
Hasil deteksi visual disimpan di: output/hasil_test (1).ppm

--- MEMULAI ANALISA PERFORMA ---
Ditemukan 1 objek ground truth.

Rata-rata IoU untuk deteksi yang cocok: 0.4333

Hasil Evaluasi (IoU Threshold = 0.5):
  - True Positives (TP) : 0
  - False Positives (FP): 1
  - False Negatives (FN): 1

  - Precision : 0.0000
  - Recall    : 0.0000
